In [2]:
import dagshub
dagshub.init(repo_owner='2654Ankit', repo_name='Next_Word_Predictor', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\AI\ML\envs\nextword\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1c52b713-d393-44b7-a919-cbbfabe5229d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e3543d3877ee8bbd96e3a4ef1315c496afc8e543df4b5675588e1dfa3e8470fe




Accessing as 2654Ankit

Initialized MLflow to track repo "2654Ankit/Next_Word_Predictor"

Repository 2654Ankit/Next_Word_Predictor initialized!

In [3]:
import tensorflow as tf

In [5]:
import pickle

In [7]:
model = pickle.load(open("./model/NextWord.pkl","rb"))

In [11]:
tokenizer = pickle.load(open("./artifacts/transformed_data/tokenizer.pkl",'rb'))

In [12]:
from dataclasses import dataclass
from pathlib import Path


In [13]:
@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data_path: Path
    all_params: dict
    mlflow_uri: str

In [14]:
from src.constants import *
from src.utils.common import read_yaml,create_directories,save_json

In [23]:
class ConfigurationManager:
    def __init__(self,
    config_filepath = CONFIG_FILE_PATH,
    params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(path_to_yaml=params_file_path)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="./model/NextWord.pkl",
            training_data_path = "./artifacts/transformed_data",
            all_params = self.params,
            mlflow_uri = "https://dagshub.com/2654Ankit/Next_Word_Predictor.mlflow")

        return eval_config


In [24]:
import mlflow.keras
from urllib.parse import urlparse

In [29]:
class ModelEvaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    @staticmethod
    def load_model(path:Path):
        return pickle.load(open("model/NextWord.pkl",'rb'))

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self.score = model.evaluate([6,1,5],[6,1,2])

    def save_score(self):
        score = {"loss":self.score[0],"accuracy":self.score[1]}

        save_json(path=Path("score.json"),data=score)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )

            if tracking_url_type_store !='file':
                mlflow.keras.log_model(self.model,"model",registered_model_name='NextWordv1')
            else:
                mlflow.keras.log_model(self.model,"model")

    




In [30]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = ModelEvaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e


[2025-01-19 11:23:14,163:INFO: common:yaml file :config\config.yaml loaded successfully]
[2025-01-19 11:23:14,165:INFO: common:yaml file :params.yaml loaded successfully]
[2025-01-19 11:23:14,167:INFO: common:created dir at : artifacts]


ValueError: in user code:

    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\engine\training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\engine\training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\engine\training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\engine\training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\AI\ML\envs\nextword\Lib\site-packages\keras\engine\input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered 